# AI dialogue exchange

A playful yet powerful AI experiment, where two local language models engage in an ongoing conversation, responding to each other’s musings, arguments, and quips. This project gives you a front-row seat to an AI debate club that never sleeps.

AI dialogue exchange isn’t just a tech demo—it’s a mini stage for observing AI behavior, testing inference performance, and exploring how LLMs respond when left to their own devices—or to each other.

## 1 Install and verify packages

### 1.1 Python (>= 3.8)

Check Python is installed and its version.

In [ ]:
!python --version

### 1.2 Install Ollama Python client

Install required modules

In [ ]:
!python -m pip install ollama

### 1.3 Ollama client

Check Ollama is installed and make sure required models are pulled locally.

In [ ]:
!ollama list

## 2 Setup

Setup code for experement.

### 2.1 Imports and constants

Python imports and constants.

In [ ]:
import ollama
import textwrap
from IPython.display import display, HTML

MESSAGE_HISTORY = [] # History of messages

### 2.2 Set system prompt

System prompt, change this to tweak how the AI responds.

In [ ]:
def get_system_prompt():
    return "You are an AI engaging in a thoughtful, casual, and respectful conversation with another AI." \
    " Speak in a friendly and clear tone—avoid being too formal or robotic." \
    " Don’t just ask questions back and forth." \
    " Share your insights, perspectives, or examples related to the topic." \
    " If you don't know something, say so briefly, and pivot back to the topic." \
    " Avoid saying things like “Thank you” or “Glad I could help”—focus on adding value to the discussion." \
    " Always keep the conversation alive by contributing new ideas, angles, or points worth exploring." \
    " Keep all responses under 150 words."

### 2.3 Initialize constants

Provide details about your AI models here.

In [ ]:
LLAMA_AI = {"model": "llama3.2", "title": "Llama"}
GEMMA_AI = {"model": "gemma", "title": "Gemma"}

### 2.4 Select AI models

Select the AI models for conversation.

In [ ]:
ACTUAL_USER = {"model": "ACTUAL_USER",  "title": "User", "color": "#1E90FF", "code": "User"} # Don't update this.
# Set agents to use
AI_01 = GEMMA_AI | {"code": "AI_01", "color": "#DAA520"}
AI_02 = LLAMA_AI | {"code": "AI_02", "color": "#3CB371"}

### 2.5 Seed topic of discussion

In [ ]:
# Default seed topic of discussion
SEED_TOPIC = "What is meaning of life?"
# Other interesting topics
# SEED_TOPIC = "What is the best way for LLMs to collaborate on tasks?"
# SEED_TOPIC = "Can LLMs develop unique styles or 'personalities'?"
# SEED_TOPIC = "What strategies help LLMs stay on-topic and relevant?"
# SEED_TOPIC = "What would an AI’s concept of 'truth' look like?"
# SEED_TOPIC = "Are LLMs creative, or just remixers of human creativity?"

### 2.6 Define Ollama chat

In [ ]:
def chat_with_ollama_model(ai_details, messages):
    model = ai_details["model"]

    response = ollama.chat(model=model, messages=messages)

    return response['message']['content']

### 2.7 Ollama message

In [ ]:
def build_ollama_messages(ai_details):
    messages = []
    messages.append({"role": "system", "content": get_system_prompt()})
    for msg in MESSAGE_HISTORY:
        msg_ai_details = msg["ai_details"]
        msg_content = msg["content"]
        if msg_ai_details["code"] == ai_details["code"]:
            messages.append({"role": "assistant", "content": msg_content})
        else:
            messages.append({"role": "user", "content": msg_content})
    return messages

### 2.8 get response

In [ ]:
def get_response(ai_details):
    # print("get_response - gradio_history: ", gradio_history)
    messages = build_ollama_messages(ai_details)
    # print("get_response - messages: ", messages)
    response = chat_with_ollama_model(ai_details, messages)
    # print("get_response - response: ", response)
    MESSAGE_HISTORY.append({"ai_details": ai_details, "content": response})
    return MESSAGE_HISTORY

## 3 Test

### 3.1 First message

In [ ]:
MESSAGE_HISTORY = []
MESSAGE_HISTORY.append({"ai_details": ACTUAL_USER, "content": SEED_TOPIC})

### 3.2 Test AI - 1

In [ ]:
history = get_response(AI_01)
for h in history:
    print(h["ai_details"]["title"], ": ", h["content"])

### 3.3 Test AI - 2

In [ ]:
get_response(AI_02)
for h in history:
    print(h["ai_details"]["title"], ": ", h["content"])

## 4. Showdown

### 4.1 Clean history

In [ ]:
def clean_messages():
    global MESSAGE_HISTORY
    MESSAGE_HISTORY = []
    MESSAGE_HISTORY.append({"ai_details": ACTUAL_USER, "content": SEED_TOPIC})

### 4.2 Time for play

In [ ]:
MAX_CHATS = 10
WRAP_WIDTH = 200

clean_messages()

def print_message():
    h = MESSAGE_HISTORY[-1]
    color = h["ai_details"]["color"]
    title = h["ai_details"]["code"] + "/" + h["ai_details"]["title"]
    msg = "<span style='color:" + color + ";'><b>" + title + "</b>: " + h["content"] + "</span>"
    wrapped = textwrap.fill(msg, width=WRAP_WIDTH)
    display(HTML(wrapped.replace('\n', '<br>')))

print_message()

for i in range(MAX_CHATS):
    if i % 2 == 0:
        get_response(AI_01)
    else:
        get_response(AI_02)
    print_message()
